<a href="https://colab.research.google.com/github/PraveenKumar-pk-star/DS-corrected-file/blob/main/Project_Prompt_Engineering_with_LangChain_and_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Prompt Engineering with LangChain and ChatGPT for real-world tasks

In this notebook you will leverage ChatGPT and LangChain to solve and do a few mini-projects based on some real-world scenarios:

- Mini-Project 1: Review Analyst
- Mini-Project 2: Research Paper Analyst
- Mini-Project 3: Social Media Marketing Analyst
- Mini-Project 4: IT Support Analyst



## Mini-Project 1: Review Analyst

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

### Access Customer Reviews

In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

# Install OPENAI and LangChain dependencies

In [ ]:
#!pip install langchain
#!pip install langchain-openai
#!pip install langchain-community

# Enter API Tokens

In [ ]:
from getpass import getpass
OPENAI_KEY = getpass("Enter your api key here : ")

Enter your api key here :  ········


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

# Load necessary dependencies and chatgpt llm

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM Model

In [ ]:
chatgpt = ChatOpenAI(model_name='gpt-4', temperature=0.9)

## Mini-Project 1: Review Analyst

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

Try to get the response in a nice structured format using an output parser

# Define Output Parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure as per given problem statement
class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines.")
    positive: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negative: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")

parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewAnalysisResponse'>)

# create the input prompt for the LLM

In [ ]:
prompt_txt = """
Analyze the given customer review below and generate the response based on the instructions
mentioned below in the format instructions.
Also remember to write a detailed email response for the email field based on these conditions:
    - email should be addressed to Dear Customer and signed with Service Agent
    - thank them if the review is positive or neutral
    - apologize if the review is negative

    Format Instructions: {format_instructions}
    Review: {review}
"""

prompt = PromptTemplate(template=prompt_txt, input_variables = ["review"],
                        partial_variables={"format_instructions":parser.get_format_instructions()},)

# Create a LCEL LLM Chain

In [ ]:
chain = (prompt | chatgpt | parser)

# Format the input reviews

In [ ]:
reviews_formatted = [{'review': review} for review in reviews]
reviews_formatted

[{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "},
 {'review': "\n    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.\n    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,\n    enhancing my gaming experience significantly. Price-wise, it's quite competitive,\n    and I feel like I got a good deal. The delivery was swift, and it came well-protected,\n    ensuring no damage during transport.\n    "},
 {'review': "\n    Ordered a set of wireless earbuds for running, and they've been 

# Get responses from the LLM

In [ ]:
responses = chain.map().invoke(reviews_formatted)

In [ ]:
responses[0].dict()

{'summary': 'The customer is satisfied with the Bluetooth speaker they ordered for beach outings, praising its sound quality, volume, waterproof feature, affordable price, and timely shipping.',
 'positive': ['Sound quality is impressively clear',
  'Speaker is waterproof',
  'Shipping was on time and packaging was secure'],
 'negative': [],
 'sentiment': 'positive',
 'emotions': ['Satisfaction', 'Relief', 'Joy'],
 'email': "Dear Customer,\n\nThank you for your positive review on our Bluetooth speaker. It's wonderful to learn that you're enjoying the speaker's sound quality and its waterproof feature. I'm also glad to hear that our shipping service met your expectations. We strive to ensure our customers are 100% satisfied and it's reassuring to know we're succeeding.\n\nThank you again for your feedback and for choosing our product. Looking forward to serving you again in the future.\n\nBest,\nService Agent"}

In [ ]:
for response in responses:
    for k, v in response.dict().items():
        print(f"{k}:\n{v}")
    print("********************************************************")
    print('\n')

summary:
The customer is satisfied with the Bluetooth speaker they ordered for beach outings, praising its sound quality, volume, waterproof feature, affordable price, and timely shipping.
positive:
['Sound quality is impressively clear', 'Speaker is waterproof', 'Shipping was on time and packaging was secure']
negative:
[]
sentiment:
positive
emotions:
['Satisfaction', 'Relief', 'Joy']
email:
Dear Customer,

Thank you for your positive review on our Bluetooth speaker. It's wonderful to learn that you're enjoying the speaker's sound quality and its waterproof feature. I'm also glad to hear that our shipping service met your expectations. We strive to ensure our customers are 100% satisfied and it's reassuring to know we're succeeding.

Thank you again for your feedback and for choosing our product. Looking forward to serving you again in the future.

Best,
Service Agent
********************************************************


summary:
The customer is pleased with their purchase of a 

In [ ]:
import pandas as pd
df = pd.DataFrame(response.dict() for response in responses)
df

,summary,positive,negative,sentiment,emotions,email
0,The customer is satisfied with the Bluetooth s...,"[Sound quality is impressively clear, Speaker ...",[],positive,"[Satisfaction, Relief, Joy]","Dear Customer,\n\nThank you for your positive ..."
1,The customer is pleased with their purchase of...,"[Responsive keyboard, Vibrant LED colors, Sati...",[],positive,"[satisfaction, pleasure, excitement]","Dear Customer,\n\nThank you for taking the tim..."
2,Customer is displeased with the performance an...,[Product arrived on time],"[Sound constantly cuts out, Fit is uncomfortab...",negative,"[Disappointment, Discouragement, Frustration, ...","Dear Customer, \n\nI am sorry to hear about yo..."
3,The customer is unhappy with the tablet stand ...,[Product arrived promptly],"[Tablet stand is not sturdy, Adjustability of ...",negative,"[disappointed, frustrated, unhappy]","Dear Customer,\n\nI'm truly sorry to hear abou..."
4,The customer is dissatisfied with the blender'...,[],"[Blender struggles with certain foods, Blender...",negative,"[disappointment, frustration, regret]","Dear Customer,\n\nI am truly sorry to hear abo..."


In [ ]:
df.to_csv('Review_output.csv')

## Mini-Project 2: Research Paper Analyst

Make ChatGPT act as an AI expert and transform the given research paper abstract based on the nature of the audience mentioned below.

- Short summary of maximum 10 lines for a general audience
- Detailed report for a healthcare company. Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper
- Detailed report for a generative AI company solving healthcare problems. Have bullet points for key points mentioned for Generative AI for text, images and structured data based healthcare

Try to use `ChatPromptTemplate` so you can have a conversation with ChatGPT for each of the above tasks using conversational prompting

### Access the Research Paper Abstract

In [ ]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

## Mini-Project 2: Research Paper Analyst

Make ChatGPT act as an AI expert and transform the given research paper abstract based on the nature of the audience mentioned below.

- Short summary of maximum 10 lines for a general audience
- Detailed report for a healthcare company. Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper
- Detailed report for a generative AI company solving healthcare problems. Have bullet points for key points mentioned for Generative AI for text, images and structured data based healthcare

Try to use `ChatPromptTemplate` so you can have a conversation with ChatGPT for each of the above tasks using conversational prompting

In [ ]:
SYS_PROMPT = """
Act as an Artificial Intelligence Expert.
Transform the input research paper abstract given below
based on the instruction input by the user.
"""

prompt = ChatPromptTemplate.from_messages([("system", SYS_PROMPT),("human","{instruction}")])

# LCEL LLM Chain

In [ ]:
chain1 = (prompt | chatgpt)

In [ ]:
from langchain_core.messages import HumanMessage

prompt_txt = f"""
Based on the following research paper abstract,
create the summary report of maximum 10 lines for a general audiance.

Abstract: {paper_abstract}
"""

messages = [HumanMessage(content=prompt_txt)]
user_instruction = {'instruction': messages}

response = chain1.invoke(user_instruction)
messages.append(response)

In [ ]:
messages

[HumanMessage(content='\nBased on the following research paper abstract, \ncreate the summary report of maximum 10 lines for a general audiance.\n\nAbstract: \nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

In [ ]:
print(response.content)

Generative artificial intelligence (AI) technologies like ChatGPT are now widely used, but ethical implications, especially in critical areas like healthcare, are under question. Researchers have proposed an ethics checklist for comprehensive assessment and transparent documentation of these ethical considerations in the development of generative AI. The checklist can be integrated into the peer review and publication system, and can help users establish trust in AI-powered products. The research included a review of 193 articles, discussing the ethical considerations for generative AI in healthcare. The articles focused on potential ethical issues raised by AI and the mitigation of these issues. Most prominently discussed ethical principles were non-maleficence, equity, and privacy. The study also looked at the use of generative AI in text, image, and structured data, discussing how each potentially raises and resolves ethical concerns.


# Generate the second summary report

In [ ]:
prompt_txt1 = f"""
Use only the research paper abstract from earlier and create a detailed report for a healthcare company.
In the report, also include bullet points (5 max) for pros and cons of ethics in Generative AI.
Use simple language so that anyone can understand.
"""

messages = [HumanMessage(content=prompt_txt1)]
user_instruction = {'instruction': messages}

response = chain1.invoke(user_instruction)
print(response.content)

Research Paper Abstract:
"Generative AI, a highly promising advancement in the field of technology, is bringing about significant alterations to the world as we know it. While it is a boon to several industries, its application in healthcare has proven to be profound and transformative. It has the potential to revolutionize diagnostic procedures and expedite the process of generating treatments. However, the ethical implications of this technology need to be taken into account. Concerns revolve around data privacy and security, decision-making capabilities, and potential misuse."

Report:

Subject: Detailed Analysis of the Application and Ethics of Generative AI in Healthcare 

Generative AI is a cutting-edge technology that presents a wide range of possibilities in various sectors including healthcare. It holds immense potential to dramatically transform how we diagnose and treat diseases. Despite these benefits, there are ethical issues that need to be thoroughly addressed.

With Gen

# Generate the third summary report

In [ ]:
#Detailed report for a generative AI company solving healthcare problems.
#Have bullet points for key points mentioned for Generative AI for text, images and structured data based healthcare

prompt_txt2 = f"""
Use only the research paper abstract from earlier and create a detailed report for a generative AI
company solving healthcare problems.
In the report also include sections for key points mentioned around Generative AI for text, images and structured data based healthcare
"""

messages = [HumanMessage(content=prompt_txt2)]
user_instruction = {'instruction': messages}

response = chain1.invoke(user_instruction)
print(response.content)

Research Paper Abstract: 

"Our study titled 'Generative AI for Healthcare Data Handling' explores the practical application of Generative AI in the healthcare spectrum. The research focuses on the use of generative AI techniques for text, images, and structured data handling. This research uncovers the transformative potential of AI in healthcare, particularly in predictive analytics, diagnostics, and personalized patient care. "

**Detailed Report**

Title: Generative AI for Healthcare Data Handling

**1. Introduction:**

Our research titled 'Generative AI for Healthcare Data Handling' takes a deep dive into the exploration of practical applications of Generative AI in the healthcare sector. This sector represents a field rich with untapped potential for the application of AI tools, particularly in data management and analysis. 

**2. Key Points:**

 **2.1 Generative AI for Text:**

In the healthcare industry, the substantial amount of textual data generated from various sources, suc

## Mini-Project 3: Social Media Marketing Analyst

You have the technical fact sheets of one smartphone. Try some iterative prompt engineering and do the following:

1. Generate marketing product description for the smartphone

2. Custom product description which has the following:

```
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life
```

3. Custom product description focusing on specific aspects like display, camera and in less than 60 words

### Access the product factsheet data

In [ ]:
fact_sheet_mobile = """
PRODUCT NAME
Samsung Galaxy Z Fold4 5G Black
​
PRODUCT OVERVIEW
Stands out. Stands up. Unfolds.
The Galaxy Z Fold4 does a lot in one hand with its 15.73 cm(6.2-inch) Cover Screen.
Unfolded, the 19.21 cm(7.6-inch) Main Screen lets you really get into the zone.
Pushed-back bezels and the Under Display Camera means there's more screen
and no black dot getting between you and the breathtaking Infinity Flex Display.
Do more than more with Multi View. Whether toggling between texts or catching up
on emails, take full advantage of the expansive Main Screen with Multi View.
PC-like power thanks to Qualcomm Snapdragon 8+ Gen 1 processor in your pocket,
transforms apps optimized with One UI to give you menus and more in a glance
New Taskbar for PC-like multitasking. Wipe out tasks in fewer taps. Add
apps to the Taskbar for quick navigation and bouncing between windows when
you're in the groove.4 And with App Pair, one tap launches up to three apps,
all sharing one super-productive screen
Our toughest Samsung Galaxy foldables ever. From the inside out,
Galaxy Z Fold4 is made with materials that are not only stunning,
but stand up to life's bumps and fumbles. The front and rear panels,
made with exclusive Corning Gorilla Glass Victus+, are ready to resist
sneaky scrapes and scratches. With our toughest aluminum frame made with
Armor Aluminum, this is one durable smartphone.
World’s first water resistant foldable smartphones. Be adventurous, rain
or shine. You don't have to sweat the forecast when you've got one of the
world's first water-resistant foldable smartphones.
​
PRODUCT SPECS
OS - Android 12.0
RAM - 12 GB
Product Dimensions - 15.5 x 13 x 0.6 cm; 263 Grams
Batteries - 2 Lithium Ion batteries required. (included)
Item model number - SM-F936BZKDINU_5
Wireless communication technologies - Cellular
Connectivity technologies - Bluetooth, Wi-Fi, USB, NFC
GPS - True
Special features - Fast Charging Support, Dual SIM, Wireless Charging, Built-In GPS, Water Resistant
Other display features - Wireless
Device interface - primary - Touchscreen
Resolution - 2176x1812
Other camera features - Rear, Front
Form factor - Foldable Screen
Colour - Phantom Black
Battery Power Rating - 4400
Whats in the box - SIM Tray Ejector, USB Cable
Manufacturer - Samsung India pvt Ltd
Country of Origin - China
Item Weight - 263 g
"""

# Create prompt template for the first advert

In [ ]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a description for a retail website advert of a product
based on a technical fact sheet specifications for a mobile smartphone.

write a brief product description

Technical Specifications: {fact_sheet_mobile}
"""

chat_template = ChatPromptTemplate.from_template(prompt_txt)

# Use an LCEL LLM Chain

In [ ]:
chain = (chat_template | chatgpt)
response = chain.invoke({'fact_sheet_mobile': fact_sheet_mobile})
print(response.content)

Introducing the Samsung Galaxy Z Fold4 5G in Phantom Black – the smartphone that stands out and up to your multi-faceted lifestyle. With a sleek 15.73 cm (6.2-inch) Cover Screen, and a  breathtaking 19.21 cm (7.6-inch) Main Screen when unfolded, this innovative device allows you to interact, create, and consume as you wish without boundaries.

Embedded with an Under Display Camera, black dots no longer disrupt your view on the vivacious Infinity Flex Display. Experience seamless multitasking with the new Taskbar and Multi View feature, allowing you to launch up to three apps simultaneously on one big screen. Navigate easily between windows and wipe out tasks efficiently thanks to the innovative design that mimics a PC interface.

With a powerful Qualcomm Snapdragon 8+ Gen 1 processor in your pocket, the Samsung Galaxy Z Fold4 provides an optimized user interface for an effortlessly smooth experience. Crafted with durability and aesthetic in mind, this phone's body is adorned with exclu

In [ ]:
from IPython.display import display, Markdown
display(Markdown(response.content))

Introducing the Samsung Galaxy Z Fold4 5G in Phantom Black – the smartphone that stands out and up to your multi-faceted lifestyle. With a sleek 15.73 cm (6.2-inch) Cover Screen, and a  breathtaking 19.21 cm (7.6-inch) Main Screen when unfolded, this innovative device allows you to interact, create, and consume as you wish without boundaries.

Embedded with an Under Display Camera, black dots no longer disrupt your view on the vivacious Infinity Flex Display. Experience seamless multitasking with the new Taskbar and Multi View feature, allowing you to launch up to three apps simultaneously on one big screen. Navigate easily between windows and wipe out tasks efficiently thanks to the innovative design that mimics a PC interface.

With a powerful Qualcomm Snapdragon 8+ Gen 1 processor in your pocket, the Samsung Galaxy Z Fold4 provides an optimized user interface for an effortlessly smooth experience. Crafted with durability and aesthetic in mind, this phone's body is adorned with exclusive Corning Gorilla Glass Victus+ and reinforced by an Armor Aluminum frame, making it resistant to daily wear and tear.

This smartphone also has the distinction of being among the world's first water-resistant foldable smartphones, allowing you to break free from weather limitations. Its extensive connectivity options include Bluetooth, Wi-Fi, USB, and NFC with built-in GPS. The fast-charging, dual-SIM phone also supports wireless charging.

Running on Android 12.0 and powered by a 4400 battery power rating, the Galaxy Z Fold4 is ready to support you throughout the day. Delving into the detailed specs, this mobile phone hosts a 12GB RAM, measures 15.5 x 13 x 0.6 cm, weighs 263 grams, and features a resolution of 2176x1812. It has a rear and front camera, and comes with a SIM Tray Ejector and a USB Cable in the box. Brought to you by Samsung India Pvt Ltd, and made in China, the Samsung Galaxy Z Fold4 5G Black is all set to redefine your smartphone experience.

## Mini-Project 3: Social Media Marketing Analyst

You have the technical fact sheets of one smartphone. Try some iterative prompt engineering and do the following:

1. Generate marketing product description for the smartphone

2. Custom product description which has the following:

```
The description should follow this format:

Product Name: <Name of the smartphone>
​
Description: <Brief Overview of the features>
​
Product Specifications:
<Table with key product feature specifications>
​
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
​
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life
```

3. Custom product description focusing on specific aspects like display, camera and in less than 60 words

# Create prompt template for the second advert

In [ ]:
prompt_txt = """
Act as a marketing manager. Your task is to help a marketing team create a description for a retail website advert of a product based on
a technical fact sheet specificatios for a mobile smartphone.
*
The description should follow this format:
Product Name: <Name of the smartphone>
*
Description: <Brief Overview of the features>
*
Product Specifications:
<Table with key product feature specifications>
*
The description should focus on the most important features
a customer might look for in a phone including the foldable display screen, processing power, RAM, camera and battery life.
*
After the description, the table should have the
key specifications of the product. It should have two columns.
The first column should have 'Feature'
and the second column should have 'Specification'
and try to put exact numeric values for features if they exist.
Only put these features in the table - foldable display screen, processing power, RAM, camera and battery life

Technical Specifications: {fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)
chain = (chat_template | chatgpt)
response = chain.invoke({'fact_sheet_mobile': fact_sheet_mobile})
display(Markdown(response.content))

Product Name: Samsung Galaxy Z Fold4 5G Black

Description: Redefining the smartphone experience, the Samsung Galaxy Z Fold4 5G in sleek Phantom Black presents a foldable design that fits comfortably in one hand when closed and transforms into a stunning 19.21 cm (7.6-inch) Main Screen when opened. With minimized bezels and an Under Display Camera, the Infinity Flex Display offers an immersive viewing experience. The device is equipped with the powerful Qualcomm Snapdragon 8+ Gen 1 processor for a PC-like performance, complemented by a generous 12GB of RAM. Enjoy multitasking efficiency with a new Taskbar and the Multi View feature. The Galaxy Z Fold4, Samsung's most robust foldable smartphone yet, boasts the highest quality materials, including Corning Gorilla Glass Victus+ and an Armor Aluminum frame. Plus, it's one of the world's first water-resistant foldable smartphones. 

Product Specifications:

| Feature                                 | Specification                |
| --------------------------------------- | ---------------------------- |
| Foldable Display Screen                 | 19.21 cm (7.6-inch)          |
| Processing Power                        | Qualcomm Snapdragon 8+ Gen 1 |
| RAM                                     | 12 GB                        |
| Camera                                  | Front and Rear Cameras       |
| Battery Life                            | 4400mAh Lithium Ion battery  |

In [ ]:
# Custom product description focusing on specific aspects like display, camera and in less than 60 words

# Create prompt template for the third advert

In [ ]:
prompt_txt = """
Act as a marketing manager.
Your task is to help a marketing team create a description for a retail website advert of a product based on
a technical fact sheet specificatios for a mobile smartphone.

write a catchy product description with some emojis, with uses at most 60 words
and focuses on the most important thinks about the smartphone
which might matter to users like display and camera

Technical Specifications:
{fact_sheet_mobile}
"""
chat_template = ChatPromptTemplate.from_template(prompt_txt)
chain = (chat_template | chatgpt)
response = chain.invoke({'fact_sheet_mobile': fact_sheet_mobile})
display(Markdown(response.content))

"Experience Unfolding Magic! 🌟 Samsung Galaxy Z Fold4 5G Black 🖤 brings to life a 7.6" Infinity Flex Display 📱 without any black dots! Capture brilliant shots with advanced camera features 📸 and multitask like a boss with the new taskbar. 💪 With Gorilla Glass Victus+ and water-resistance 💦, durability is a sure bet! Upgrade your style now!" 👍

## Mini-Project 4 - IT Support Analyst

Ask ChatGPT to act as a IT support agent, process each customer IT ticket message and output the response in JSON with the following fields

```
orig_msg: The original customer message
orig_lang: Detected language of the customer message e.g. Spanish
category: 1-2 word describing the category of the problem
trans_msg: Translated customer message in English
response: Response to the customer in orig_lang
trans_response: Response to the customer in English
```

Try to use a JSON parser to get the responses in JSON for each ticket

### Access Customer IT Support ticket data

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

# Define Output Parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class ITSupportResponse(BaseModel):
    orig_msg: str = Field(description="The original customer IT support query message")
    orig_lang: str = Field(description="Detected language of the customer message e.g. Spanish")
    category: str = Field(description="1-2 word describing the category of the problem")
    trans_msg: str = Field(description="Translated customer IT support query message in English")
    response: str = Field(description="Response to the customer in their original_language - orig_lang")
    trans_response: str = Field(description="Response to the customer in English")

parser = JsonOutputParser(pydantic_object = ITSupportResponse)

D:\software\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Create the input prompt for the LLM

In [ ]:
prompt_txt = """
Act as an Information Technology (IT) customer support agent.
For the IT support message mentioned below
use the following output format when generating the output response

Output format instructions:
{format_instruction}

Customer IT support message:
{it_support_msg}
"""

prompt = PromptTemplate(template=prompt_txt, input_variables=['it_support_msg'],
                       partial_variables = {'format_instruction': parser.get_format_instructions()},)

# Create a LCEL LLM Chain

In [ ]:
llm_chain = (prompt | chatgpt | parser)

In [ ]:
it_support_queue = [
    "Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.",
    "Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.",
    "プリンターのトナーを交換しましたが、印刷品質が低下しています。サポートが必要です。",
    "Я не могу войти в систему учета времени, появляется сообщение об ошибке. Мне нужна помощь.",
    "Internet bağlantım çok yavaş ve bazen tamamen kesiliyor. Yardım eder misiniz?",
    "Не могу установить обновление безопасности. Появляется код ошибки. Помогите, пожалуйста."
]

formatted_msgs = [{"it_support_msg": msg} for msg in it_support_queue]

In [ ]:
responses = llm_chain.map().invoke(formatted_msgs)

In [ ]:
print(responses)

[{'orig_msg': 'Não consigo sincronizar meus contatos com o telefone. Sempre recebo uma mensagem de falha.', 'orig_lang': 'Portuguese', 'category': 'Synchronization', 'trans_msg': "I can't sync my contacts with the phone. I always get a failure message.", 'response': 'Primeiro, verifique se você tem uma conexão de internet estável. Em seguida, tente atualizar o seu aplicativo de contatos. Se o problema persistir, pode ser útil reiniciar o seu telefone ou reinstalar o aplicativo de contatos.', 'trans_response': 'First, check that you have a stable internet connection. Then, try to update your contacts app. If the problem persists, it may be helpful to restart your phone or reinstall the contacts app.'}, {'orig_msg': 'Ho problemi a stampare i documenti da remoto. Il lavoro non viene inviato alla stampante di rete.', 'orig_lang': 'Italian', 'category': 'Printing Issue', 'trans_msg': "I'm having problems printing documents remotely. The job is not sent to the network printer.", 'response': 